In [1]:
import subprocess
import multiprocessing
import os
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook as tqdm
import time
from scipy.stats import ttest_rel

In [2]:
neighbourhoods = ["--transpose", "--exchange", "--insert", "--tei", "--tie"]
def run_fssp(instance, init=0, improvement=0, neighbourhood=0):
    try:
        init = "--random-init" if not init else "--srz"
        improvement = "--first" if not improvement else "--best"
        neighbourhood = neighbourhoods[neighbourhood]
        command = "./fssp " + " ".join([instance, init, improvement, neighbourhood])
        beg_time = time.time()
        output = subprocess.check_output(command, shell=True, stderr=subprocess.DEVNULL)
        solution, score = output.decode('utf-8').split('\n')[:2]
        return command, [int(x) for x in solution.split(" ") if x], int(score), time.time() - beg_time
    except Exception as e:
        print(e)
        return command, [], 0


In [3]:
def run_instance(instance):
    res = []
    for neighbourhood in [0, 1, 2]:
        for init in [0, 1]:
            for improvement in [0, 1]:
                res.append(run_fssp(instance, init, improvement, neighbourhood))
    for neighbourhood in [3, 4]:
        res.append(run_fssp(instance, 0, 0, neighbourhood))
    return res


In [4]:
def all_instances_names():
    for filename in os.listdir("../instances"):
        if filename != "bestSolutions.txt":
            yield os.path.join("../instances/", filename)

In [5]:
instances = list(all_instances_names())
pool = multiprocessing.Pool(multiprocessing.cpu_count())
res = list(tqdm(pool.imap_unordered(run_instance, instances), total=len(instances)))        

In [35]:
import itertools
a = list(itertools.chain(*res))
len(a)

840

In [36]:
processed = [
    {
        "instance": s.split()[1].split("/")[-1],
        "n_jobs": len(sol),
        "init": "srz" if "srz" in s else "random",
        "pivot": "best" if "best" in s else "first",
        "neighbourhood": "exchange" if "exchange" in s else "transpose" if "transpose" in s else "insert" if "insert"in s else "tie" if "tie" in s else "tei",
        "score": score,
        "time": time
    }
    for s, sol, score, time in a
]
df = pd.DataFrame(processed)

In [37]:
df

,init,instance,n_jobs,neighbourhood,pivot,score,time
0,random,50_20_28,50,transpose,first,725848,0.043515
1,random,50_20_28,50,transpose,best,724189,0.028999
2,srz,50_20_28,50,transpose,first,565683,0.019947
3,srz,50_20_28,50,transpose,best,565683,0.023180
4,random,50_20_28,50,exchange,first,566459,0.904839
5,random,50_20_28,50,exchange,best,572886,0.183649
6,srz,50_20_28,50,exchange,first,565197,0.047767
7,srz,50_20_28,50,exchange,best,565197,0.040680
8,random,50_20_28,50,insert,first,575218,0.777843
9,random,50_20_28,50,insert,best,583491,0.420746


In [9]:
best = pd.read_csv("../instances/bestSolutions.txt")
best.columns = ["instance", "best_solution"]
best.instance = best.instance.str.strip()
best.best_solution = best.best_solution.astype(int)
best.sample()

,instance,best_solution
25,100_20_26,1565920


In [38]:
merged = pd.merge(df, best, how='left', on='instance')
merged.sample()

,init,instance,n_jobs,neighbourhood,pivot,score,time,best_solution
56,random,50_20_05,50,transpose,first,842494,0.022593,653748


In [39]:
merged['percentage_deviation'] = 100 * (merged.score - merged.best_solution) / merged.best_solution

In [40]:
exercise1 = merged[(merged.neighbourhood != "tei") & (merged.neighbourhood != "tie")]
print(len(exercise1))
exercise1.sample()

720


,init,instance,n_jobs,neighbourhood,pivot,score,time,best_solution,percentage_deviation
711,srz,50_20_12,50,insert,best,596857,0.081508,584199,2.166727


# Exercise 1.1

In [41]:
exercise1_groupby_algos = exercise1.groupby(['init', 'pivot', 'neighbourhood', 'n_jobs'])
exercise1_groupby_algos.mean()[['percentage_deviation']]

percentage_deviation
init   pivot neighbourhood n_jobs                      
random best  exchange      50                  4.385601
                           100                 4.667283
             insert        50                  8.170156
                           100                10.903816
             transpose     50                 32.736274
                           100                42.109242
       first exchange      50                  1.965092
                           100                 1.715059
             insert        50                  6.265681
                           100                 7.724756
             transpose     50                 31.101046
                           100                41.205408
srz    best  exchange      50                  2.787518
                           100                 3.432429
             insert        50                  2.811639
                           100                 3.511336
             transpose     50                  3.701844
                           100                 4.587164
       first exchange      50                  2.858113
                           100                 3.119183
             insert        50                  2.669091
                           100                 3.202606
             transpose     50                  3.680069
                           100                 4.580643

In [144]:
exercise1_groupby_algos.mean()[['time']].merge(exercise1_groupby_algos.mean()[['percentage_deviation']], left_index=True, right_index=True)

time  percentage_deviation
init   pivot neighbourhood n_jobs                                 
random best  exchange      50       0.380826              4.385601
                           100      5.774185              4.667283
             insert        50       0.463165              8.170156
                           100      7.013268             10.903816
             transpose     50       0.032342             32.736274
                           100      0.149474             42.109242
       first exchange      50       1.137775              1.965092
                           100     29.003526              1.715059
             insert        50       1.454173              6.265681
                           100     35.099521              7.724756
             transpose     50       0.042230             31.101046
                           100      0.151754             41.205408
srz    best  exchange      50       0.098529              2.787518
                           100      1.330605              3.432429
             insert        50       0.106048              2.811639
                           100      1.404951              3.511336
             transpose     50       0.026746              3.701844
                           100      0.084794              4.587164
       first exchange      50       0.106017              2.858113
                           100      1.688183              3.119183
             insert        50       0.136853              2.669091
                           100      1.915914              3.202606
             transpose     50       0.025735              3.680069
                           100      0.083676              4.580643

# Exercise 1.2 (ttest)

## Exercise 1.2.(i) (which initial solution is preferable)

In [43]:
exercise1_algos = pd.DataFrame(exercise1.groupby(['init', 'pivot', 'neighbourhood']).groups).T
exercise1_algos_scores = exercise1_algos.applymap(lambda x: exercise1.loc[x].percentage_deviation)
exercise1_algos_scores['values'] = pd.concat([exercise1_algos_scores[x] for x in exercise1_algos_scores.columns], axis=1).values.tolist()
exercise1_algos_scores = exercise1_algos_scores.drop(exercise1_algos_scores.columns.difference(['values']), axis=1)
exercise1_algos_times = exercise1_algos.applymap(lambda x: exercise1.loc[x].time)
exercise1_algos_times['values'] = pd.concat([exercise1_algos_times[x] for x in exercise1_algos_times.columns], axis=1).values.tolist()
exercise1_algos_times = exercise1_algos_times.drop(exercise1_algos_times.columns.difference(['values']), axis=1)

def student_two_columns(df, left, right):
    print(left, right)
    df = df['values'][[left, right]]
    return df.apply(lambda x: ttest_rel(x[left], x[right]), axis=1) \
             .rename(columns={left: 't-statistic', right: 'p-value'})

In [44]:
student_two_columns(exercise1_algos_scores.unstack(level=0), 'random', 'srz')

random srz


t-statistic       p-value
best  exchange     10.476357  4.369954e-15
      insert       19.474311  2.261227e-27
      transpose    37.461813  7.836271e-43
first exchange     -9.202500  5.260806e-13
      insert       16.750731  4.282623e-24
      transpose    34.188782  1.364952e-40

## Exercise 1.2.(ii) (which pivoting rule generates better quality solutions and which is faster)

In [45]:
student_two_columns(exercise1_algos_scores.unstack(level=1), 'best', 'first')

best first


t-statistic       p-value
random exchange     20.873300  6.287508e-29
       insert        9.874594  4.116715e-14
       transpose     7.331677  7.418797e-10
srz    exchange      1.496189  1.399349e-01
       insert        2.801594  6.866970e-03
       transpose     1.079051  2.849567e-01

In [46]:
exercise1_algos_scores.unstack(level=1).applymap(np.mean)

values           
                       best      first
random exchange    4.526442   1.840076
       insert      9.536986   6.995218
       transpose  37.422758  36.153227
srz    exchange    3.109973   2.988648
       insert      3.161487   2.935848
       transpose   4.144504   4.130356

In [47]:
student_two_columns(exercise1_algos_times.unstack(level=1), 'best', 'first')

best first


t-statistic       p-value
random exchange     -5.500323  8.576936e-07
       insert       -5.929607  1.691923e-07
       transpose    -1.182591  2.417150e-01
srz    exchange     -2.095175  4.045711e-02
       insert       -3.671649  5.205013e-04
       transpose     0.639531  5.249530e-01

In [48]:
exercise1_algos_times.unstack(level=1).applymap(np.mean)

values           
                      best      first
random exchange   3.077505  15.070651
       insert     3.738216  18.276847
       transpose  0.090908   0.096992
srz    exchange   0.714567   0.897100
       insert     0.755499   1.026384
       transpose  0.055770   0.054705

## Exercise 1.2.(iii) (which neighborhood generates better quality solution and what computation time is required to reach local optima)

In [49]:
student_two_columns(exercise1_algos_scores.unstack(level=2), 'exchange', 'insert')

exchange insert


t-statistic       p-value
random best    -13.951116  2.432585e-20
       first   -21.633866  9.676010e-30
srz    best     -0.611820  5.430072e-01
       first     0.511523  6.108939e-01

In [50]:
student_two_columns(exercise1_algos_scores.unstack(level=2), 'exchange', 'transpose')

exchange transpose


t-statistic       p-value
random best    -36.015470  7.267839e-42
       first   -34.908654  4.226097e-41
srz    best     -9.571960  1.289837e-13
       first   -10.442561  4.951419e-15

In [51]:
student_two_columns(exercise1_algos_scores.unstack(level=2), 'insert', 'transpose')

insert transpose


t-statistic       p-value
random best    -35.444221  1.791755e-41
       first   -32.976053  1.035748e-39
srz    best    -10.546375  3.374830e-15
       first   -11.682677  5.520169e-17

In [52]:
exercise1_algos_scores.unstack(level=2).applymap(np.mean)

values                     
              exchange    insert  transpose
random best   4.526442  9.536986  37.422758
       first  1.840076  6.995218  36.153227
srz    best   3.109973  3.161487   4.144504
       first  2.988648  2.935848   4.130356

In [53]:
student_two_columns(exercise1_algos_times.unstack(level=2), 'exchange', 'insert')

exchange insert


t-statistic   p-value
random best     -1.458233  0.150079
       first    -1.896952  0.062732
srz    best     -0.320431  0.749774
       first    -0.673858  0.503033

In [54]:
student_two_columns(exercise1_algos_times.unstack(level=2), 'exchange', 'transpose')

exchange transpose


t-statistic       p-value
random best      5.973594  1.430502e-07
       first     5.751502  3.329326e-07
srz    best      5.223762  2.399115e-06
       first     4.885012  8.265864e-06

In [55]:
student_two_columns(exercise1_algos_times.unstack(level=2), 'insert', 'transpose')

insert transpose


t-statistic       p-value
random best      5.953036  1.547281e-07
       first     6.193529  6.158958e-08
srz    best      5.451697  1.028835e-06
       first     6.009458  1.247321e-07

In [56]:
exercise1_algos_times.unstack(level=2).applymap(np.mean)

values                     
               exchange     insert transpose
random best    3.077505   3.738216  0.090908
       first  15.070651  18.276847  0.096992
srz    best    0.714567   0.755499  0.055770
       first   0.897100   1.026384  0.054705

# Exercise 2

In [57]:
exercise2 = merged[(merged.neighbourhood == "tei") | (merged.neighbourhood == "tie")]
print(len(exercise2))
exercise2.sample()

120


,init,instance,n_jobs,neighbourhood,pivot,score,time,best_solution,percentage_deviation
783,random,100_20_19,100,tie,first,1742579,0.670166,1675710,3.990488


In [58]:
exercise2_groupby_algos = exercise2.groupby(['init', 'pivot', 'neighbourhood'])
exercise2_groupby_algos.mean()[['percentage_deviation']]

percentage_deviation
init   pivot neighbourhood                      
random first tei                        2.550408
             tie                        2.745331

In [145]:
exercise2_groupby_algos.mean()[['time']].merge(exercise2_groupby_algos.mean()[['percentage_deviation']], left_index=True, right_index=True)

time  percentage_deviation
init   pivot neighbourhood                                
random first tei            1.149409              2.550408
             tie            0.797948              2.745331

In [71]:
exercise2_for_comparison = merged[(merged.neighbourhood.isin(['exchange', 'insert', 'tei', 'tie'])) & (merged.init == 'random') & (merged['pivot'] == 'first')]

In [72]:
exercise2_for_comparison

,init,instance,n_jobs,neighbourhood,pivot,score,time,best_solution,percentage_deviation
4,random,50_20_28,50,exchange,first,566459,0.904839,549809,3.028324
8,random,50_20_28,50,insert,first,575218,0.777843,549809,4.621423
12,random,50_20_28,50,tei,first,559822,0.094374,549809,1.821178
13,random,50_20_28,50,tie,first,561436,0.158919,549809,2.114734
18,random,50_20_30,50,exchange,first,518086,0.624086,509210,1.743092
22,random,50_20_30,50,insert,first,544535,1.121129,509210,6.937216
26,random,50_20_30,50,tei,first,519301,0.059992,509210,1.981697
27,random,50_20_30,50,tie,first,516550,0.100769,509210,1.441449
32,random,50_20_02,50,exchange,first,637929,0.945312,622342,2.504571
36,random,50_20_02,50,insert,first,673296,1.222909,622342,8.187460


In [114]:
def exercise2_compare_percentage_deviation(df, left, right):
    return df[df.neighbourhood == left].set_index('instance').percentage_deviation / \
           df[df.neighbourhood == right].set_index('instance').percentage_deviation

exercise2_compare_percentage_deviation(exercise2_for_comparison, 'tei', 'insert').mean()

0.38514710928022777

In [110]:
exercise2_compare_percentage_deviation(exercise2_for_comparison, 'tei', 'exchange').mean()

1.5737456238628005

In [111]:
exercise2_compare_percentage_deviation(exercise2_for_comparison, 'tie', 'insert').mean()

0.41297412544301676

In [112]:
exercise2_compare_percentage_deviation(exercise2_for_comparison, 'tie', 'exchange').mean()

1.6819576081793821

In [115]:
def exercise2_compare_time(df, left, right):
    return df[df.neighbourhood == left].set_index('instance').time / \
           df[df.neighbourhood == right].set_index('instance').time

exercise2_compare_time(exercise2_for_comparison, 'tei', 'insert').mean()

0.081733898872211358

In [116]:
exercise2_compare_time(exercise2_for_comparison, 'tei', 'exchange').mean()

0.10948147230867943

In [117]:
exercise2_compare_time(exercise2_for_comparison, 'tie', 'insert').mean()

0.066355872790064122

In [118]:
exercise2_compare_time(exercise2_for_comparison, 'tie', 'exchange').mean()

0.086926703366620409

In [142]:
from collections import defaultdict
exercise2_student_percentage_deviation = defaultdict(lambda: {})
for i in range(len(neighbourhoods)):
    for j in range(i + 1, len(neighbourhoods)):
        neigh1 = neighbourhoods[i][2:]
        neigh2 = neighbourhoods[j][2:]
        left = merged[(merged.neighbourhood == neigh1) & (merged.init == 'random') & (merged['pivot'] == 'first')]
        right = merged[(merged.neighbourhood == neigh2) & (merged.init == 'random') & (merged['pivot'] == 'first')]
        exercise2_student_percentage_deviation[neigh1,neigh2] = ttest_rel(left.percentage_deviation, right.percentage_deviation)
exercise2_student_percentage_deviation = pd.DataFrame(exercise2_student_percentage_deviation).T
exercise2_student_percentage_deviation.index = exercise2_student_percentage_deviation.index.tolist()
exercise2_student_percentage_deviation.columns = ['t-statistic', 'p-value']
exercise2_student_percentage_deviation

,t-statistic,p-value
"(exchange, insert)",-21.633866,9.676010e-30
"(exchange, tei)",-6.448070,2.308079e-08
"(exchange, tie)",-7.665743,2.013937e-10
"(insert, tei)",19.813530,9.323705e-28
"(insert, tie)",18.267205,5.813199e-26
"(tei, tie)",-2.157681,3.503523e-02
"(transpose, exchange)",34.908654,4.226097e-41
"(transpose, insert)",32.976053,1.035748e-39
"(transpose, tei)",35.543228,1.530907e-41
"(transpose, tie)",35.515420,1.600011e-41
